In [1]:
import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

In [2]:
import rasa_nlu
import rasa_core
import spacy

In [3]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

#loading the nlu training samples
training_data = load_data("data/nlu.md")
trainer = Trainer(config.load('config.yml'))

#training the nlu
interpreter = trainer.train(training_data)
model_directory = trainer.persist("models/nlu", fixed_model_name='current')


INFO:rasa_nlu.training_data.loading:Training data format of data/nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 47 (11 distinct intents)
	- Found intents: 'goodbye', 'after_service', 'Grateful', 'no_1', 'no_3', 'no_4', 'Greetings', 'no_6', 'no_5', 'no_2', 'Reason'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

INFO:rasa_nlu.utils.spacy_utils:Trying to load spacy model with name 'en_core_web_md'
INFO:rasa_nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en_core_web_md'.
INFO:rasa_nlu.model:Starting to train component SpacyNLP
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component SpacyTokenizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component CRFEntityExtractor
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component SpacyFeaturizer
INFO:rasa_nlu.model:Finished training compo

Fitting 2 folds for each of 6 candidates, totalling 12 fits


INFO:rasa_nlu.model:Successfully saved model into 'C:\Users\nakal\Documents\Nakala\MyRasa\models\nlu\default\current'


In [4]:
import json
def pprint(o):
    print(json.dumps(o,indent=2))
    
pprint(interpreter.parse('4')) 

{
  "intent": {
    "name": "goodbye",
    "confidence": 0.3567359844556009
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "goodbye",
      "confidence": 0.3567359844556009
    },
    {
      "name": "Reason",
      "confidence": 0.3369077500001384
    },
    {
      "name": "after_service",
      "confidence": 0.09957714032951863
    },
    {
      "name": "Greetings",
      "confidence": 0.09880321233188932
    },
    {
      "name": "Grateful",
      "confidence": 0.06873303424118642
    },
    {
      "name": "no_1",
      "confidence": 0.008521943976150165
    },
    {
      "name": "no_2",
      "confidence": 0.006811536705985087
    },
    {
      "name": "no_5",
      "confidence": 0.006426370268169395
    },
    {
      "name": "no_3",
      "confidence": 0.005957256307751568
    },
    {
      "name": "no_4",
      "confidence": 0.005889359129668655
    }
  ],
  "text": "4"
}


In [5]:
# Import the policies and agent
from rasa_core.policies import FallbackPolicy, MemoizationPolicy,KerasPolicy
from rasa_core.agent import Agent

# Initialize the model with `domain.yml`
agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy()])

# loading our  training dialogues from `stories.md` 
training_data = agent.load_data('data/stories.md')


# Training the model
agent.train(training_data)

agent.persist('models/dialogue')

INFO:apscheduler.scheduler:Scheduler started
Processed Story Blocks: 100%|██████████| 6/6 [00:00<00:00, 177.26it/s, # trackers=19]
Processed actions: 23it [00:00, 453.50it/s, # examples=23]
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
INFO:rasa_core.policies.keras_policy:Fitting model with 361 total samples and a validation split of 0.1


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 34)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8576      
_________________________________________________________________
dense (Dense)                (None, 19)                627       
_________________________________________________________________
activation (Activation)      (None, 19)                0         
Total params: 9,203
Trainable params: 9,203
Non-trainable params: 0
_________________________________________________________________


Instructions for updating:
Use tf.cast instead.


Epoch 1/100
361/361 [==============================] - 2s 6ms/sample - loss: 2.8278 - acc: 0.3684
Epoch 2/100
361/361 [==============================] - 0s 205us/sample - loss: 2.6153 - acc: 0.4986
Epoch 3/100
361/361 [==============================] - 0s 199us/sample - loss: 2.3405 - acc: 0.5014
Epoch 4/100
361/361 [==============================] - 0s 188us/sample - loss: 2.0378 - acc: 0.5014
Epoch 5/100
361/361 [==============================] - 0s 199us/sample - loss: 1.8426 - acc: 0.5014
Epoch 6/100
361/361 [==============================] - 0s 185us/sample - loss: 1.7911 - acc: 0.5014
Epoch 7/100
361/361 [==============================] - 0s 194us/sample - loss: 1.7285 - acc: 0.5014
Epoch 8/100
361/361 [==============================] - 0s 197us/sample - loss: 1.6807 - acc: 0.5014
Epoch 9/100
361/361 [==============================] - 0s 197us/sample - loss: 1.6365 - acc: 0.5014
Epoch 10/100
361/361 [==============================] - 0s 188us/sample - loss: 1.6136 - acc: 0.5014
E

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory models/dialogue exists and contains old model files. All files will be overwritten.
INFO:rasa_core.agent:Persisted model to 'C:\Users\nakal\Documents\Nakala\MyRasa\models\dialogue'


In [6]:
#Starting the Bot
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



INFO:rasa_nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en_core_web_md'.


In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'
